# TCN Classification
<hr>




## Load the library

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: Config option `use_jedi` not recognized by `IPCompleter`.
  del sys.path[0]


In [3]:
tf.config.list_physical_devices('GPU') 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load the Dataset

In [ ]:
corpus = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sentence_version_1.csv')
corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(8679, 4)


,Unnamed: 0,sentence,label,split
0,1,van huy,0,train
1,2,end intern,0,train
2,3,education,0,train
3,4,vietnam korea university information communica...,1,train
4,5,project school website sell bakery,0,train
...,...,...,...,...
8674,8675,programming tool mysql workbench intellij ide,1,train
8675,8676,coffee shop project management,1,train
8676,8677,description manage coffee shop activity add in...,1,train
8677,8678,technology run time environment microsoft wind...,1,train


In [ ]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8679 entries, 0 to 8678
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  8679 non-null   int64 
 1   sentence    8679 non-null   object
 2   label       8679 non-null   int64 
 3   split       8679 non-null   object
dtypes: int64(2), object(2)
memory usage: 271.3+ KB


In [ ]:
corpus.groupby(by=['split', 'label']).count()

Unnamed: 0  sentence
split label                      
test  0            1057      1057
      1             743       743
train 0            3945      3945
      1            2934      2934

In [ ]:
# Separate the sentences and the labels
# Separate the sentences and the labels for training and testing
train_x = list(corpus[corpus.split=='train'].sentence)
train_y = np.array(corpus[corpus.split=='train'].label)
print(len(train_x))
print(len(train_y))

test_x = list(corpus[corpus.split=='test'].sentence)
test_y = np.array(corpus[corpus.split=='test'].label)
print(len(test_x))
print(len(test_y))

6879
6879
1800
1800


In [ ]:
train_x[100]

'design database'

<!--## Split Dataset-->

# Data Preprocessing
<hr>

Preparing data for word embedding, especially for pre-trained word embedding like Word2Vec or GloVe, __don't use standard preprocessing steps like stemming or stopword removal__. Compared to our approach on cleaning the text when doing word count based feature extraction (e.g. TFIDF) such as removing stopwords, stemming etc, now we will keep these words as we do not want to lose such information that might help the model learn better.

__Tomas Mikolov__, one of the developers of Word2Vec, in _word2vec-toolkit: google groups thread., 2015_, suggests only very minimal text cleaning is required when learning a word embedding model. Sometimes, it's good to disconnect
In short, what we will do is:
- Puntuations removal
- Lower the letter case
- Tokenization

The process above will be handled by __Tokenizer__ class in TensorFlow

- <b>One way to choose the maximum sequence length is to just pick the length of the longest sentence in the training set.</b>

In [ ]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

# Separate the sentences and the labels
train_x = list(corpus[corpus.split=='train'].sentence)
train_y = np.array(corpus[corpus.split=='train'].label)
test_x = list(corpus[corpus.split=='test'].sentence)
test_y = np.array(corpus[corpus.split=='test'].label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(train_x)

print("Example of sentence: ", train_x[4])

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(train_x)
max_len = max_length(training_sequences)

print('Into a sequence of int:', training_sequences[4])

# Pad the sequence to have the same size
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[11])

Example of sentence:  project school website sell bakery
Into a sequence of int: [2, 139, 14, 171, 1451]
Into a padded sequence: [  2 139  14 171 743 198   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]


In [ ]:
# See the first 10 words in the vocabulary

word_index = tokenizer.word_index
for i, word in enumerate(word_index):
    print(word, word_index.get(word))
    if i==9:
        break
vocab_size = len(word_index)+1
print(vocab_size)

<UNK> 1
project 2
technology 3
management 4
web 5
experience 6
java 7
test 8
work 9
html 10
4321


# Model 1: Embedding Random
<hr>

## TCN Model

Now, we will build Temporal Convolutional Network (CNN) models to classify encoded documents as either positive or negative.

The model takes inspiration from https://github.com/philipperemy/keras-tcn and https://www.kaggle.com/christofhenkel/temporal-convolutional-network

__Arguments__
`TCN(nb_filters=64, kernel_size=2, nb_stacks=1, dilations=[1, 2, 4, 8, 16, 32], padding='causal', use_skip_connections=False, dropout_rate=0.0, return_sequences=True, activation='relu', kernel_initializer='he_normal', use_batch_norm=False, **kwargs)`

- `nb_filters`: Integer. The number of filters to use in the convolutional layers. Would be similar to units for LSTM.
- `kernel_size`: Integer. The size of the kernel to use in each convolutional layer.
- `dilations`: List. A dilation list. Example is: [1, 2, 4, 8, 16, 32, 64].
- `nb_stacks`: Integer. The number of stacks of residual blocks to use.
- `padding`: String. The padding to use in the convolutions. 'causal' for a causal network (as in the original implementation) and - `'same' for a non-causal network.
- `use_skip_connections`: Boolean. If we want to add skip connections from input to each residual block.
- `return_sequences`: Boolean. Whether to return the last output in the output sequence, or the full sequence.
- `dropout_rate`: Float between 0 and 1. Fraction of the input units to drop.
- `activation`: The activation used in the residual blocks o = activation(x + F(x)).
- `kernel_initializer`: Initializer for the kernel weights matrix (Conv1D).
- `use_batch_norm`: Whether to use batch normalization in the residual layers or not.
- `kwargs`: Any other arguments for configuring parent class Layer. For example "name=str", Name of the model. Use unique names when using multiple TCN.

Now, we will define our TCN model as follows:
- One TCN layer with 100 filters, kernel size 1-6, and relu and tanh activation function;
- Dropout size = 0.5;
- Optimizer: Adam (The best learning algorithm so far)
- Loss function: binary cross-entropy (suited for binary classification problem)

**Note**: 
- The whole purpose of dropout layers is to tackle the problem of over-fitting and to introduce generalization to the model. Hence it is advisable to keep dropout parameter near 0.5 in hidden layers. 
- https://missinglink.ai/guides/keras/keras-conv1d-working-1d-convolutional-neural-networks-keras/

In [ ]:
!pip install keras-tcn

     |████████████████████████████████| 679 kB 8.3 MB/s 


In [ ]:
from tcn import TCN, tcn_full_summary
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import concatenate, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.models import Model

def define_model(kernel_size = 3, activation='relu', input_dim = None, output_dim=300, max_length = None ):
    
    inp = Input( shape=(max_length,))
    x = Embedding(input_dim=input_dim, output_dim=output_dim, input_length=max_length)(inp)
    x = SpatialDropout1D(0.1)(x)
    
    x = TCN(128,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn1')(x)
    x = TCN(64,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn2')(x)
    
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(16, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(2, activation="softmax")(conc)    

    model = Model(inputs=inp, outputs=outp)
    model.compile( loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [ ]:
model_0 = define_model(input_dim=1000, max_length=100)
model_0.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 300)     300000      input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d (SpatialDropo (None, 100, 300)     0           embedding[0][0]                  
__________________________________________________________________________________________________
tcn1 (TCN)                      (None, 100, 128)     400256      spatial_dropout1d[0][0]          
______________________________________________________________________________________________

In [ ]:
# tcn_full_summary(model_0)

In [ ]:
# class myCallback(tf.keras.callbacks.Callback):
#     # Overide the method on_epoch_end() for our benefit
#     def on_epoch_end(self, epoch, logs={}):
#         if (logs.get('accuracy') > 0.93):
#             print("\nReached 93% accuracy so cancelling training!")
#             self.model.stop_training=True


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=20, verbose=2, 
                                             mode='auto', restore_best_weights=True)

## Train and Test the Model

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/driver')
nb_path = '/content/notebooks'

from google.colab import drive
drive.mount('/content/drive')

# Install only once. Tomorrow, you can skip this.
!pip install --target=$nb_path helpers

Mounted at /content/driver
Mounted at /content/drive
  Using cached helpers-0.2.0-py3-none-any.whl (2.3 kB)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu', 'tanh']
filters = 100
kernel_sizes = [5,6]
# 1,2,3,4

columns = ['Activation', 'Filters', 'Acc']
record = pd.DataFrame(columns = columns)

# Separate the sentences and the labels
train_x = list(corpus[corpus.split=='train'].sentence)
train_y = np.array(corpus[corpus.split=='train'].label)
test_x = list(corpus[corpus.split=='test'].sentence)
test_y = np.array(corpus[corpus.split=='test'].label)

exp = 0
acc_max = 0

for activation in activations:

    for kernel_size in kernel_sizes:
        
        exp+=1
        print('-------------------------------------------')
        print('Training {}: {} activation, {} kernel size.'.format(exp, activation, kernel_size))
        print('-------------------------------------------')
        
        # encode data using
        # Cleaning and Tokenization
        tokenizer = Tokenizer(oov_token=oov_tok)
        tokenizer.fit_on_texts(train_x)

        # Turn the text into sequence
        training_sequences = tokenizer.texts_to_sequences(train_x)
        test_sequences = tokenizer.texts_to_sequences(test_x)

        max_len = max_length(training_sequences)

        # Pad the sequence to have the same size
        Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
        Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

        word_index = tokenizer.word_index
        vocab_size = len(word_index)+1

        # Define the input shape
        model = define_model(kernel_size, activation, input_dim=vocab_size, max_length=max_len)

        # Train the model and initialize test accuracy with 0
        acc = 0
        while(acc<0.7):

            history = model.fit(Xtrain, train_y, batch_size=50, epochs=100, verbose=1, 
                      callbacks=[callbacks], validation_data=(Xtest, test_y))

            # evaluate the model
            loss, acc = model.evaluate(Xtest, test_y, verbose=0)
            print('Test Accuracy: {}'.format(acc*100))

            if (acc<0.6):
                print('The model suffered from local minimum. Retrain the model!')
                model = define_model(kernel_size, activation, input_dim=vocab_size, max_length=max_len)

            else:
                # # plot_train_history(history, x_ticks_vertical=True)
                # # summarize history for accuracy
                # plt.plot(history.history['accuracy'])
                # plt.plot(history.history['val_accuracy'])
                # plt.title('model accuracy')
                # plt.ylabel('accuracy')
                # plt.xlabel('epoch')
                # plt.legend(['train', 'test'], loc='upper left')
                # plt.show()
                # # summarize history for loss
                # plt.plot(history.history['loss'])
                # plt.plot(history.history['val_loss'])
                # plt.title('model loss')
                # plt.ylabel('loss')
                # plt.xlabel('epoch')
                # plt.legend(['train', 'test'], loc='upper left')
                # plt.show()

                # y_pred = model.predict(Xtest)
                # print(confusion_matrix(test_y,y_pred))
                # print(classification_report(test_y,y_pred))
                # print(accuracy_score(test_y, y_pred))

                if acc > acc_max:
                    acc_max = acc
                    model.save('/content/drive/MyDrive/Colab Notebooks/model_best.h5')
                print('Done!')

        parameters = [activation, kernel_size]
        entries = parameters + [acc]

        temp = pd.DataFrame([entries], columns=columns)
        record = record.append(temp, ignore_index=True)
        print()
        print(record)
        print()

print('model co do chinh xac cao nhat:', acc_max)

-------------------------------------------
Training 1: relu activation, 5 kernel size.
-------------------------------------------
Epoch 1/100
138/138 [==============================] - 42s 53ms/step - loss: 0.5819 - accuracy: 0.6619 - val_loss: 0.2474 - val_accuracy: 0.9039
Epoch 2/100
138/138 [==============================] - 6s 41ms/step - loss: 0.2824 - accuracy: 0.8868 - val_loss: 0.2735 - val_accuracy: 0.8889
Epoch 3/100
138/138 [==============================] - 6s 42ms/step - loss: 0.2040 - accuracy: 0.9206 - val_loss: 0.2835 - val_accuracy: 0.8889
Epoch 4/100
138/138 [==============================] - 6s 42ms/step - loss: 0.1432 - accuracy: 0.9443 - val_loss: 0.3952 - val_accuracy: 0.8761
Epoch 5/100
138/138 [==============================] - 6s 42ms/step - loss: 0.1151 - accuracy: 0.9583 - val_loss: 0.5096 - val_accuracy: 0.8678
Epoch 6/100
138/138 [==============================] - 5s 39ms/step - loss: 0.0976 - accuracy: 0.9637 - val_loss: 0.4856 - val_accuracy: 0.8650
Epo

## Summary

In [ ]:
record.sort_values(by='Acc', ascending=False)

,Activation,Filters,Acc
2,tanh,5,0.923889
3,tanh,6,0.921111
1,relu,6,0.905556
0,relu,5,0.903889


In [ ]:
report = record.sort_values(by='Acc', ascending=False)
report = report.to_excel('TCN_TREC.xlsx', sheet_name='random')

## Test model 

In [ ]:
CV_test_path = '/content/[Fresher] Đỗ Quang Phước - Java.pdf.txt.csv'
df = pd.read_csv(CV_test_path)
df.head(10)
test_cv = list(df.Sentence)
test_sequences = tokenizer.texts_to_sequences(test_cv)
max_len = max_length(training_sequences)
CVtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
predict = model.predict(CVtest, verbose=0)
yhat_probs = np.argmax(predict, axis=1)
predict_data = pd.DataFrame(predict , columns=["U", "K"])
predict_data.to_csv('Predict_LePhuocHoang_CV_model1.csv')
df.head(10)

,Sentence
0,education
1,codegym hue
2,java web backend development
3,aptech hue
4,ltv international
5,09/2020 04/2021
6,03/2019 09/2019
7,06/2018 hien tai
8,quang phuoc
9,fresher java developer


In [ ]:
yhat_probs

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0])

In [ ]:
count = 0
for i in range(len(test_cv)):  
    if (yhat_probs[i]==0):
        print(test_cv[i],' ---------> Unknow')     
    elif (yhat_probs[i]==1):
        count += 1
        print(test_cv[i],' ---------> Knowleadge') 
print('Dem duoc: ',count ,' tu thuoc Knowleaged')

education  ---------> Unknow
codegym hue  ---------> Unknow
java web backend development  ---------> Knowleadge
aptech hue  ---------> Unknow
ltv international  ---------> Unknow
09/2020  04/2021  ---------> Unknow
03/2019  09/2019  ---------> Unknow
06/2018  hien tai  ---------> Unknow
quang phuoc  ---------> Unknow
fresher java developer  ---------> Knowleadge
work experience  ---------> Unknow
codegym bootcamp hue  ---------> Unknow
student  ---------> Unknow
15/12/2001  ---------> Unknow
nam  ---------> Unknow
037 9494 687  ---------> Unknow
doquangphuoc9999@gmail.com  ---------> Unknow
huong toan huong tra thua  ---------> Unknow
thien hue  ---------> Unknow
fb.com/quang.phuoc1512  ---------> Unknow
career goal  ---------> Unknow
want    fresher java  ---------> Knowleadge
developer  ---------> Unknow
year  stick  ---------> Unknow
junior    45 year  ---------> Unknow
senior  ---------> Unknow
skill  ---------> Unknow
html  ---------> Knowleadge
javascript  ---------> Knowleadge
g

In [ ]:
knowledge = ''
for i in range(len(test_cv)):
  if (yhat_probs[i]==1):
    knowledge += ' ' + test_cv[i]

print(knowledge)

 java web backend development fresher java developer want    fresher java html javascript mysql postgresql listen  music learn   use html css  javascript  create basic website access java language database mysql / postgresql ajax  build backend application jsp / servlet technology spring framework spring mvc spring boot spring security know   manage  divide work  product backlog kanban scrum efficient work management write clean code learn  object orient programming oop use basic html write clean code work management  teamwork skill complete   web page    html css java core language familiar  objectoriente programming oop  java java ee know java webservlet jsp familiar  database mysql design execute query  mysql module 4 4 week  160 hour complete  simple online sale website   team  3 write  spring mvc combine learn  jquery jquery ajax  asynchronous handle complete  website   coffee shop management   team  3  spring boot plan  manage job   kanban company codegym vietnam jsc company code

# Model 2: Word2Vec Static

__Using and updating pre-trained embeddings__
* In this part, we will create an Embedding layer in Tensorflow Keras using a pre-trained word embedding called Word2Vec 300-d tht has been trained 100 bilion words from Google News.
* In this part,  we will leave the embeddings fixed instead of updating them (dynamic).

1. __Load `Word2Vec` Pre-trained Word Embedding__

In [ ]:
!brew install wget

!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

/bin/bash: brew: command not found
--2021-07-30 20:19:45--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.40.182
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.40.182|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  39.4MB/s    in 50s     

2021-07-30 20:20:35 (31.4 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [ ]:
!gzip -d GoogleNews-vectors-negative300.bin.gzy

gzip: GoogleNews-vectors-negative300.bin.gzy.gz: No such file or directory


In [ ]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('/content/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
word2vec.word_vec('framework')

array([-0.13085938,  0.03881836,  0.07177734,  0.15136719, -0.16699219,
       -0.10546875,  0.04150391, -0.09375   ,  0.04467773, -0.15332031,
       -0.10253906,  0.02758789,  0.0534668 , -0.23144531, -0.375     ,
        0.09814453,  0.00527954,  0.13769531, -0.14550781, -0.21972656,
        0.0559082 ,  0.1171875 ,  0.03295898,  0.33007812,  0.1171875 ,
        0.09863281,  0.05175781,  0.16015625, -0.00805664, -0.21777344,
       -0.36914062, -0.41601562,  0.0546875 ,  0.14453125,  0.11865234,
        0.04248047, -0.14648438, -0.09960938,  0.25195312,  0.00628662,
        0.01879883, -0.05908203, -0.03833008,  0.2578125 ,  0.07666016,
       -0.09521484, -0.18847656, -0.25195312, -0.13769531,  0.18261719,
        0.10791016, -0.12792969, -0.1953125 , -0.12792969,  0.17871094,
        0.06542969, -0.17382812, -0.33203125, -0.06787109, -0.27734375,
       -0.20117188, -0.03979492,  0.09716797,  0.12890625, -0.06445312,
        0.17382812, -0.12792969,  0.05493164, -0.04907227, -0.12

2. __Check number of training words present in Word2Vec__

In [ ]:
def training_words_in_word2vector(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    vocab_size = len(word_to_index) + 1
    count = 0
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            count+=1
            
    return print('Found {} words present from {} training vocabulary in the set of pre-trained word vector'.format(count, vocab_size))

In [ ]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
training_words_in_word2vector(word2vec, word_index)

Found 1193 words present from 1474 training vocabulary in the set of pre-trained word vector


2. __Define a `pretrained_embedding_layer` function__

In [ ]:
emb_mean = word2vec.vectors.mean()
emb_std = word2vec.vectors.std()

In [ ]:
from tensorflow.keras.layers import Embedding

def pretrained_embedding_matrix(word_to_vec_map, word_to_index, emb_mean, emb_std):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    np.random.seed(2021)
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_size = len(word_to_index) + 1
    # define dimensionality of your pre-trained word vectors (= 300)
    emb_dim = word_to_vec_map.word_vec('handsome').shape[0]
    
    # initialize the matrix with generic normal distribution values
    embed_matrix = np.random.normal(emb_mean, emb_std, (vocab_size, emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            embed_matrix[idx] = word_to_vec_map.get_vector(word)
            
    return embed_matrix

In [ ]:
# Test the function
w_2_i = {'<UNK>': 1, 'framework': 2, 'java': 3, 'python': 4 }
em_matrix = pretrained_embedding_matrix(word2vec, w_2_i, emb_mean, emb_std)
em_matrix

array([[ 0.19468211,  0.08648376, -0.05924511, ..., -0.16683994,
        -0.09975549, -0.08595189],
       [-0.13509196, -0.07441947,  0.15388953, ..., -0.05400787,
        -0.13156594, -0.05996158],
       [-0.13085938,  0.03881836,  0.07177734, ..., -0.07080078,
        -0.07324219,  0.00463867],
       [-0.13574219, -0.19042969, -0.37304688, ...,  0.01867676,
         0.06542969,  0.29882812],
       [ 0.27734375,  0.12402344, -0.40820312, ...,  0.49023438,
        -0.21679688, -0.00622559]])

## TCN Model

In [ ]:
def define_model_2(kernel_size = 3, activation='relu', input_dim = None, 
                   output_dim=300, max_length = None, emb_matrix = None):
    
    inp = Input( shape=(max_length,))
    x = Embedding(input_dim=input_dim, 
                  output_dim=output_dim, 
                  input_length=max_length,
                  # Assign the embedding weight with word2vec embedding marix
                  weights = [emb_matrix],
                  # Set the weight to be not trainable (static)
                  trainable = False)(inp)
    
    x = SpatialDropout1D(0.1)(x)
    
    x = TCN(128,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn1')(x)
    x = TCN(64,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn2')(x)
    
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(16, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(3, activation="softmax")(conc)    

    model = Model(inputs=inp, outputs=outp)
    model.compile( loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [ ]:
model_0 = define_model_2( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "model_26"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_27 (InputLayer)           [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_26 (Embedding)        (None, 100, 300)     300000      input_27[0][0]                   
__________________________________________________________________________________________________
spatial_dropout1d_26 (SpatialDr (None, 100, 300)     0           embedding_26[0][0]               
__________________________________________________________________________________________________
tcn1 (TCN)                      (None, 100, 128)     400256      spatial_dropout1d_26[0][0]       
___________________________________________________________________________________________

## Train and Test the Model

In [ ]:
# class myCallback(tf.keras.callbacks.Callback):
#     # Overide the method on_epoch_end() for our benefit
#     def on_epoch_end(self, epoch, logs={}):
#         if (logs.get('accuracy') >= 0.9):
#             print("\nReached 90% accuracy so cancelling training!")
#             self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=20, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [ ]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
filters = 100
kernel_sizes = [1,2,3,4,5,6]
emb_mean = emb_mean
emb_std = emb_std

columns = ['Activation', 'Filters', 'Acc']
record2 = pd.DataFrame(columns = columns)

# Separate the sentences and the labels
train_x = list(corpus[corpus.split=='train'].sentence)
train_y = np.array(corpus[corpus.split=='train'].label)
test_x = list(corpus[corpus.split=='test'].sentence)
test_y = np.array(corpus[corpus.split=='test'].label)

exp = 0

for activation in activations:

    for kernel_size in kernel_sizes:
        
        exp+=1
        print('-------------------------------------------')
        print('Training {}: {} activation, {} kernel size.'.format(exp, activation, kernel_size))
        print('-------------------------------------------')
        
        # encode data using
        # Cleaning and Tokenization
        tokenizer = Tokenizer(oov_token=oov_tok)
        tokenizer.fit_on_texts(train_x)

        # Turn the text into sequence
        training_sequences = tokenizer.texts_to_sequences(train_x)
        test_sequences = tokenizer.texts_to_sequences(test_x)

        max_len = max_length(training_sequences)

        # Pad the sequence to have the same size
        Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
        Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

        word_index = tokenizer.word_index
        vocab_size = len(word_index)+1

        emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)

        # Define the input shape
        model = define_model_2(kernel_size, activation, input_dim=vocab_size, 
                             max_length=max_len, emb_matrix=emb_matrix)

        # Train the model and initialize test accuracy with 0
        acc = 0
        while(acc<0.6):

            history = model.fit(Xtrain, train_y, batch_size=50, epochs=100, verbose=1, 
                      callbacks=[callbacks], validation_data=(Xtest, test_y))

            # evaluate the model
            loss, acc = model.evaluate(Xtest, test_y, verbose=0)
            print('Test Accuracy: {}'.format(acc*100))

            if (acc<0.6):
                print('The model suffered from local minimum. Retrain the model!')
                model = define_model_2(kernel_size, activation, input_dim=vocab_size, 
                                       max_length=max_len, emb_matrix=emb_matrix)
            else:
                # # summarize history for accuracy
                # plt.plot(history.history['accuracy'])
                # plt.plot(history.history['val_accuracy'])
                # plt.title('model accuracy')
                # plt.ylabel('accuracy')
                # plt.xlabel('epoch')
                # plt.legend(['train', 'test'], loc='upper left')
                # plt.show()
                # # summarize history for loss
                # plt.plot(history.history['loss'])
                # plt.plot(history.history['val_loss'])
                # plt.title('model loss')
                # plt.ylabel('loss')
                # plt.xlabel('epoch')
                # plt.legend(['train', 'test'], loc='upper left')
                # plt.show()
                print('Done!')

        parameters = [activation, kernel_size]
        entries = parameters + [acc]

        temp = pd.DataFrame([entries], columns=columns)
        record2 = record2.append(temp, ignore_index=True)
        print()
        print(record2)
        print()

-------------------------------------------
Training 1: relu activation, 1 kernel size.
-------------------------------------------
Epoch 1/100
29/29 [==============================] - 3s 29ms/step - loss: 0.9496 - accuracy: 0.5462 - val_loss: 0.7211 - val_accuracy: 0.6685
Epoch 2/100
29/29 [==============================] - 0s 12ms/step - loss: 0.7058 - accuracy: 0.6741 - val_loss: 0.6243 - val_accuracy: 0.7528
Epoch 3/100
29/29 [==============================] - 0s 14ms/step - loss: 0.5737 - accuracy: 0.7622 - val_loss: 0.4667 - val_accuracy: 0.8090
Epoch 4/100
29/29 [==============================] - 0s 13ms/step - loss: 0.4478 - accuracy: 0.8238 - val_loss: 0.4272 - val_accuracy: 0.8315
Epoch 5/100
29/29 [==============================] - 0s 12ms/step - loss: 0.3611 - accuracy: 0.8608 - val_loss: 0.3936 - val_accuracy: 0.8539
Epoch 6/100
29/29 [==============================] - 0s 13ms/step - loss: 0.2712 - accuracy: 0.8972 - val_loss: 0.4719 - val_accuracy: 0.8511
Epoch 7/100
29/2

## Summary

In [ ]:
record2.sort_values(by='Acc', ascending=False)

,Activation,Filters,Acc
5,relu,6,0.870786
1,relu,2,0.867977
3,relu,4,0.867977
0,relu,1,0.859551
4,relu,5,0.859551
2,relu,3,0.848315


In [ ]:
report = record2.sort_values(by='Acc', ascending=False)
report = report.to_excel('TCN_TREC_2.xlsx', sheet_name='static')

## Test model 2


In [ ]:
CV_test_path = '/content/drive/MyDrive/Colab Notebooks/LePhuocHoang_CV.csv'
df = pd.read_csv(CV_test_path)
df.head(10)
test_cv = list(df.Sentence)
test_sequences = tokenizer.texts_to_sequences(test_cv)
max_len = max_length(training_sequences)
CVtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
predict = model.predict(CVtest, verbose=0)
yhat_probs = np.argmax(predict, axis=1)
predict_data = pd.DataFrame(predict , columns=["K", "A", "S"])
predict_data.to_csv('Predict_LePhuocHoang_CV_model2.csv')
for i in range(len(test_cv)):  
    if (yhat_probs[i]==0)and(predict[i][0] >= 0.95):
        print(test_cv[i],' ---------> Knowledge')     
    elif (yhat_probs[i]==1)and(predict[i][1] >= 0.95):
        print(test_cv[i],' ---------> Attitude') 
    elif (yhat_probs[i]==2)and(predict[i][2] >= 0.95):
        print(test_cv[i],' ---------> Skill')
    else: 
        print(test_cv[i],' ---------> Unknow')  

le phuoc hoang  ---------> Knowledge
frontend developer  ---------> Knowledge
date  birth  ---------> Unknow
18/10/1999  ---------> Unknow
gender  ---------> Unknow
male  ---------> Unknow
phone  ---------> Unknow
email  ---------> Knowledge
lephuochoang98tcv@gmail.com  ---------> Unknow
address  ---------> Unknow
danang vietnam  ---------> Knowledge
website  ---------> Knowledge
https//www.linkedin.com  ---------> Unknow
sociable  quick learn  hope  quickly integrate   work  try well  complete  ---------> Attitude
assign task  order  gain  experience  ---------> Unknow
objective  ---------> Unknow
education  ---------> Skill
august 2017  present  ---------> Unknow
skill summary  ---------> Skill
project  ---------> Unknow
danang university  technology  ---------> Knowledge
major information technology  ---------> Knowledge
4th year student  ---------> Unknow
language  ---------> Unknow
html css javascript java c  ---------> Knowledge
framework  ---------> Unknow
reactjs redux  -------

# Model 3: Word2Vec - Dynamic

* In this part,  we will fine tune the embeddings while training (dynamic).

## TCN Model


In [ ]:
def define_model_3(kernel_size = 3, activation='relu', input_dim = None, 
                   output_dim=300, max_length = None, emb_matrix = None):
    
    inp = Input( shape=(max_length,))
    x = Embedding(input_dim=input_dim, 
                  output_dim=output_dim, 
                  input_length=max_length,
                  # Assign the embedding weight with word2vec embedding marix
                  weights = [emb_matrix],
                  # Set the weight to be not trainable (static)
                  trainable = True)(inp)
    
    x = SpatialDropout1D(0.1)(x)
    
    x = TCN(128,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn1')(x)
    x = TCN(64,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn2')(x)
    
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(16, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(3, activation="softmax")(conc)    

    model = Model(inputs=inp, outputs=outp)
    model.compile( loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [ ]:
model_0 = define_model_3( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "model_33"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_34 (InputLayer)           [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_33 (Embedding)        (None, 100, 300)     300000      input_34[0][0]                   
__________________________________________________________________________________________________
spatial_dropout1d_33 (SpatialDr (None, 100, 300)     0           embedding_33[0][0]               
__________________________________________________________________________________________________
tcn1 (TCN)                      (None, 100, 128)     400256      spatial_dropout1d_33[0][0]       
___________________________________________________________________________________________

## Train and Test the Model

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=20, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [ ]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
filters = 100
kernel_sizes = [1,2,3,4,5,6]
emb_mean = emb_mean
emb_std = emb_std

columns = ['Activation', 'Filters', 'Acc']
record3 = pd.DataFrame(columns = columns)

# Separate the sentences and the labels
train_x = list(corpus[corpus.split=='train'].sentence)
train_y = np.array(corpus[corpus.split=='train'].label)
test_x = list(corpus[corpus.split=='test'].sentence)
test_y = np.array(corpus[corpus.split=='test'].label)

exp = 0

for activation in activations:

    for kernel_size in kernel_sizes:
        
        exp+=1
        print('-------------------------------------------')
        print('Training {}: {} activation, {} kernel size.'.format(exp, activation, kernel_size))
        print('-------------------------------------------')
        
        # encode data using
        # Cleaning and Tokenization
        tokenizer = Tokenizer(oov_token=oov_tok)
        tokenizer.fit_on_texts(train_x)

        # Turn the text into sequence
        training_sequences = tokenizer.texts_to_sequences(train_x)
        test_sequences = tokenizer.texts_to_sequences(test_x)

        max_len = max_length(training_sequences)

        # Pad the sequence to have the same size
        Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
        Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

        word_index = tokenizer.word_index
        vocab_size = len(word_index)+1

        emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)

        # Define the input shape
        model = define_model_3(kernel_size, activation, input_dim=vocab_size, 
                             max_length=max_len, emb_matrix=emb_matrix)

        # Train the model and initialize test accuracy with 0
        acc = 0
        while(acc<0.6):

            model.fit(Xtrain, train_y, batch_size=50, epochs=200, verbose=1, 
                      callbacks=[callbacks], validation_data=(Xtest, test_y))

            # evaluate the model
            loss, acc = model.evaluate(Xtest, test_y, verbose=0)
            print('Test Accuracy: {}'.format(acc*100))

            if (acc<0.6):
                print('The model suffered from local minimum. Retrain the model!')
                model = define_model_3(kernel_size, activation, input_dim=vocab_size, 
                                       max_length=max_len, emb_matrix=emb_matrix)
            else:
                # # summarize history for accuracy
                # plt.plot(history.history['accuracy'])
                # plt.plot(history.history['val_accuracy'])
                # plt.title('model accuracy')
                # plt.ylabel('accuracy')
                # plt.xlabel('epoch')
                # plt.legend(['train', 'test'], loc='upper left')
                # plt.show()
                # # summarize history for loss
                # plt.plot(history.history['loss'])
                # plt.plot(history.history['val_loss'])
                # plt.title('model loss')
                # plt.ylabel('loss')
                # plt.xlabel('epoch')
                # plt.legend(['train', 'test'], loc='upper left')
                # plt.show()
                print('Done!')

        parameters = [activation, kernel_size]
        entries = parameters + [acc]

        temp = pd.DataFrame([entries], columns=columns)
        record3 = record3.append(temp, ignore_index=True)
        print()
        print(record3)
        print()

-------------------------------------------
Training 1: relu activation, 1 kernel size.
-------------------------------------------
Epoch 1/200
29/29 [==============================] - 3s 33ms/step - loss: 0.9706 - accuracy: 0.5483 - val_loss: 0.7008 - val_accuracy: 0.6854
Epoch 2/200
29/29 [==============================] - 1s 19ms/step - loss: 0.6636 - accuracy: 0.7042 - val_loss: 0.5989 - val_accuracy: 0.6798
Epoch 3/200
29/29 [==============================] - 0s 16ms/step - loss: 0.4847 - accuracy: 0.8105 - val_loss: 0.4613 - val_accuracy: 0.8371
Epoch 4/200
29/29 [==============================] - 0s 16ms/step - loss: 0.3353 - accuracy: 0.8713 - val_loss: 0.4022 - val_accuracy: 0.8567
Epoch 5/200
29/29 [==============================] - 0s 17ms/step - loss: 0.2024 - accuracy: 0.9336 - val_loss: 0.4132 - val_accuracy: 0.8511
Epoch 6/200
29/29 [==============================] - 0s 17ms/step - loss: 0.1599 - accuracy: 0.9497 - val_loss: 0.4186 - val_accuracy: 0.8567
Epoch 7/200
29/2

## Summary

In [ ]:
record3.sort_values(by='Acc', ascending=False)

,Activation,Filters,Acc
4,relu,5,0.887640
2,relu,3,0.882023
1,relu,2,0.879214
3,relu,4,0.876405
0,relu,1,0.870786
5,relu,6,0.870786


In [ ]:
report = record3.sort_values(by='Acc', ascending=False)
report = report.to_excel('TCN_TREC_3.xlsx', sheet_name='dynamic')

## Test model 3

In [ ]:
CV_test_path = '/content/drive/MyDrive/Colab Notebooks/LePhuocHoang_CV.csv'
df = pd.read_csv(CV_test_path)
df.head(10)
test_cv = list(df.Sentence)
test_sequences = tokenizer.texts_to_sequences(test_cv)
max_len = max_length(training_sequences)
CVtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
predict = model.predict(CVtest, verbose=0)
yhat_probs = np.argmax(predict, axis=1)
print(yhat_probs)
print()
predict_data = pd.DataFrame(predict , columns=["K", "A", "S"])
predict_data.to_csv('Predict_LePhuocHoang_CV_model3.csv')
for i in range(len(test_cv)):  
    if (yhat_probs[i]==0)and(predict[i][0] >= 0.95):
        print(test_cv[i],' ---------> Knowledge')     
    elif (yhat_probs[i]==1)and(predict[i][1] >= 0.95):
        print(test_cv[i],' ---------> Attitude') 
    elif (yhat_probs[i]==2)and(predict[i][2] >= 0.95):
        print(test_cv[i],' ---------> Skill')
    else: 
        print(test_cv[i],' ---------> Unknow')  

[2 0 2 2 2 2 2 0 2 2 0 0 2 1 2 0 2 2 2 2 0 0 0 0 0 2 0 0 0 2 0 0 1 1 0 0 0
 0 0 2 0 0 0 0 0 2 2]

le phuoc hoang  ---------> Skill
frontend developer  ---------> Knowledge
date  birth  ---------> Unknow
18/10/1999  ---------> Skill
gender  ---------> Unknow
male  ---------> Unknow
phone  ---------> Skill
email  ---------> Knowledge
lephuochoang98tcv@gmail.com  ---------> Skill
address  ---------> Unknow
danang vietnam  ---------> Knowledge
website  ---------> Knowledge
https//www.linkedin.com  ---------> Skill
sociable  quick learn  hope  quickly integrate   work  try well  complete  ---------> Attitude
assign task  order  gain  experience  ---------> Unknow
objective  ---------> Knowledge
education  ---------> Unknow
august 2017  present  ---------> Unknow
skill summary  ---------> Skill
project  ---------> Unknow
danang university  technology  ---------> Knowledge
major information technology  ---------> Knowledge
4th year student  ---------> Knowledge
language  ---------> Knowledge
